In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/bike-sharing-dataset")

print("Path to dataset files:", path)

c:\Users\pablo.lopezmartinez\AppData\Local\miniconda3\envs\bigdata\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 286k/286k [00:00<00:00, 951kB/s]

Extracting model files...
Path to dataset files: C:\Users\pablo.lopezmartinez\.cache\kagglehub\datasets\lakshmi25npathi\bike-sharing-dataset\versions\1
